
# 🚀 Especialidad ML — Runner Colab (IMDb LSTM + MNIST GAN)

Este notebook ejecuta **ambos scripts** de tu proyecto directamente desde **Google Drive**:

- `scripts/lstm_imdb.py` → *Clasificación de sentimiento con LSTM (IMDb)*  
- `scripts/gan_mnist.py` → *GAN básica para generar dígitos MNIST*

> **Cómo usar**
> 1) Monta Drive y fija `PROJECT_ROOT` a la carpeta de TU proyecto.  
> 2) (Opcional) Ajusta los argumentos de cada script.  
> 3) Ejecuta las celdas de cada sección.


## 1) Dependencias (TensorFlow + utilidades)

In [ ]:

# Colab ya viene con TF, pero fijamos versiones compatibles si fuese necesario.
import tensorflow as tf, numpy as np
print("TensorFlow:", tf.__version__)
print("NumPy:", np.__version__)
# Si hay incompatibilidades, puedes descomentar la siguiente línea:
# !pip -q install -U 'tensorflow==2.15.0' 'numpy<2.3' matplotlib pillow


## 2) Montar Drive y fijar ruta del proyecto

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

# ⚠️ EDITA ESTA RUTA: debe apuntar a la carpeta RAÍZ de tu proyecto en Drive
# Ejemplos:
# '/content/drive/MyDrive/EspecialidadML/Modulo7Clase2MarcoParra'
# '/content/drive/Shareddrives/MiUnidad/Modulo7Clase2MarcoParra'
PROJECT_ROOT = '/content/drive/MyDrive/EspecialidadML/Modulo7Clase2MarcoParra'  # <-- CAMBIA AQUÍ

import os, sys, glob
assert os.path.isdir(PROJECT_ROOT), f'No existe PROJECT_ROOT: {PROJECT_ROOT}'
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)
print('Working dir:', os.getcwd())

# Mostrar estructura de archivos principal
!find . -maxdepth 3 -type f -print | sed -n '1,200p'


## 3) Comprobación de GPU

In [ ]:

import tensorflow as tf
print("TensorFlow:", tf.__version__)
print("Dispositivos GPU:", tf.config.list_physical_devices('GPU'))



## 4) Ejecutar IMDb LSTM

Ejecuta tu script **`scripts/lstm_imdb.py`**.  
Ajusta argumentos si lo deseas (épocas, batch, bidireccional, etc.).


In [ ]:

# Args por defecto del script (ajusta si quieres):
IMDB_EPOCHS = 10
IMDB_BATCH  = 64
IMDB_MAXLEN = 250
IMDB_OUT    = 'outputs_imdb'

# Ejecutar por módulo:
!python -m scripts.lstm_imdb --epochs {IMDB_EPOCHS} --batch-size {IMDB_BATCH}


### Ver resultados IMDb (curvas, QC, matriz de confusión)

In [ ]:

import os, glob, json, pandas as pd
from IPython.display import Image, display

# Detectar el último run por nombre de archivo
histories = sorted(glob.glob(os.path.join('outputs_imdb', '*_history.json')))
assert histories, "No se encontró history.json en outputs_imdb"
last_history = histories[-1]
run_prefix = last_history.replace('_history.json','')
curves_png = run_prefix + '_curves.png'
qc_md      = run_prefix + '_history_qc.md'
cm_csv     = run_prefix + '_confusion_matrix.csv'
test_json  = run_prefix + '_test_report.json'

print("Run base:", run_prefix)
if os.path.isfile(curves_png):
    display(Image(filename=curves_png))
else:
    print("No se encontró figura de curvas:", curves_png)

# Mostrar test report
if os.path.isfile(test_json):
    with open(test_json,'r') as f:
        print("\nTEST REPORT:", json.load(f))
else:
    print("No existe:", test_json)

# Mostrar matriz de confusión si existe como CSV
if os.path.isfile(cm_csv):
    import pandas as pd
    print("\nMatriz de confusión:")
    df_cm = pd.read_csv(cm_csv, header=None)
    df_cm.columns = ['Pred:Neg', 'Pred:Pos']
    df_cm.index   = ['Real:Neg', 'Real:Pos']
    display(df_cm)
else:
    print("No existe:", cm_csv)

print("\nArchivos en outputs_imdb/")
for p in sorted(glob.glob(os.path.join('outputs_imdb', '*'))):
    print("-", os.path.basename(p))



## 5) Ejecutar MNIST GAN

Ejecuta tu script **`scripts/gan_mnist.py`**.  
Guarda pérdidas e imágenes cada ciertos pasos.


In [ ]:

# Args por defecto del script (ajusta si tu script los acepta):
GAN_OUT   = 'outputs_gan'
GAN_STEPS = 3000
GAN_SEED  = 42

# Ejecutar por módulo (si tu script acepta flags, añádelos aquí).
# --steps 3000 --out-dir outputs_gan --seed 42
!python -m scripts.gan_mnist


### Ver resultados GAN (pérdidas y muestras generadas)

In [ ]:

import os, glob
from IPython.display import Image, display

loss_png = os.path.join('outputs_gan', 'gan_losses.png')
if os.path.isfile(loss_png):
    display(Image(filename=loss_png))
else:
    if os.path.isfile('gan_losses.png'):
        display(Image(filename='gan_losses.png'))
    else:
        print("No se encontró gan_losses.png")

samples = sorted(glob.glob(os.path.join('outputs_gan', 'samples_step_*.png')))
if not samples:
    samples = sorted(glob.glob('samples_step_*.png'))
if samples:
    print(f"Mostrando {min(3,len(samples))} figuras (de {len(samples)})...")
    for p in samples[-3:]:
        display(Image(filename=p))
else:
    print("No se encontraron samples_step_*.png")

print("\nArchivos en outputs_gan/")
if os.path.isdir('outputs_gan'):
    for p in sorted(glob.glob(os.path.join('outputs_gan', '*'))):
        print("-", os.path.basename(p))
else:
    print("No existe outputs_gan/ (verifica dónde guarda tu script)")
